# Proyecto del Día 18 - Analizar una Base de Datos de Perros

Hoy vamos a realizar un proyecto muy entretenido, que consiste en conseguir un dataset sobre razas de perros, que se encuentra alojada en el sitio [The Dog Api](https://thedogapi.com/).

Tu misión consiste en lograr obtener un dataset que contiene información sobre razas de perro ("breeds"), y crear visualizaciones que permitan responder algunas preguntas sobre la **expectativa de vida**, y el **temperamento** más frecuente entre todas las **razas**.


### Preguntas a responder
1. ¿Cuál es la esperanza de vida (en años) más frecuente entre todas las razas de perro?
2. ¿Cuál es el temperamento más frecuente entre todas las razas de perros?


### Pasos sugeridos para llegar hasta la respuesta
1. Descarga las **librerías** necesarias.
2. Explora el sitio [https://thedogapi.com/](https://thedogapi.com/) para identificar dónde y cómo se describe el funcionamiento de su API. Identifica la **API específica** con información sobre **razas** ("breeds"). Una vez que lo hagas, implementa la API y guarda esos datos en un **DataFrame** de Pandas.
*Nota: como todo sitio nuevo al que enfrentes, deberás resolver desafíos particulares. En este caso presta atención a las instrucciones que mencionan algo sobre cómo usar los mismos protocolos que se utilizan en la API de gatos de la misma organización*.
3. Realiza un **análisis exploratorio** para conocer la estrucutra y calidad de tus datos.
4. Implementa la **limpieza de datos** que consideres necesaria.
5. Crea los **gráficos** que creas necesarios para poder visualizar con claridad la respuesta a las preguntas.

Te deseo mucho aprendizaje, pero sobre todo, mucha diversión.
¡Adelante!

## Primeras acciones exploratorias

In [1]:
import os
import requests
from typing import Any

import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from dotenv import load_dotenv

In [2]:
load_dotenv('.env')

True

In [3]:
API_KEY: str = os.environ['API_KEY']

In [4]:
api_reference: str = 'api.thedogapi.com'
base_url: str = f'https://{api_reference}/v1/breeds'
params: dict[str, str] = {'api_key': API_KEY}

In [5]:
response: requests.Response = requests.get(base_url, params=params)
data: list[dict[str, Any]] = response.json()
df: pd.DataFrame = pd.DataFrame(data)
df.head()

,weight,height,id,name,bred_for,breed_group,life_span,temperament,origin,reference_image_id,image,country_code,description,history
0,"{'imperial': '6 - 13', 'metric': '3 - 6'}","{'imperial': '9 - 11.5', 'metric': '23 - 29'}",1,Affenpinscher,"Small rodent hunting, lapdog",Toy,10 - 12 years,"Stubborn, Curious, Playful, Adventurous, Activ...","Germany, France",BJa4kxc4X,"{'id': 'BJa4kxc4X', 'width': 1600, 'height': 1...",NaN,NaN,NaN
1,"{'imperial': '50 - 60', 'metric': '23 - 27'}","{'imperial': '25 - 27', 'metric': '64 - 69'}",2,Afghan Hound,Coursing and hunting,Hound,10 - 13 years,"Aloof, Clownish, Dignified, Independent, Happy","Afghanistan, Iran, Pakistan",hMyT4CDXR,"{'id': 'hMyT4CDXR', 'width': 606, 'height': 38...",AG,NaN,NaN
2,"{'imperial': '44 - 66', 'metric': '20 - 30'}","{'imperial': '30', 'metric': '76'}",3,African Hunting Dog,A wild pack animal,NaN,11 years,"Wild, Hardworking, Dutiful",,rkiByec47,"{'id': 'rkiByec47', 'width': 500, 'height': 33...",NaN,NaN,NaN
3,"{'imperial': '40 - 65', 'metric': '18 - 29'}","{'imperial': '21 - 23', 'metric': '53 - 58'}",4,Airedale Terrier,"Badger, otter hunting",Terrier,10 - 13 years,"Outgoing, Friendly, Alert, Confident, Intellig...","United Kingdom, England",1-7cgoZSh,"{'id': '1-7cgoZSh', 'width': 645, 'height': 43...",NaN,NaN,NaN
4,"{'imperial': '90 - 120', 'metric': '41 - 54'}","{'imperial': '28 - 34', 'metric': '71 - 86'}",5,Akbash Dog,Sheep guarding,Working,10 - 12 years,"Loyal, Independent, Intelligent, Brave",,26pHT3Qk7,"{'id': '26pHT3Qk7', 'width': 600, 'height': 47...",NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   weight              172 non-null    object
 1   height              172 non-null    object
 2   id                  172 non-null    int64 
 3   name                172 non-null    object
 4   bred_for            151 non-null    object
 5   breed_group         156 non-null    object
 6   life_span           172 non-null    object
 7   temperament         168 non-null    object
 8   origin              5 non-null      object
 9   reference_image_id  172 non-null    object
 10  image               172 non-null    object
 11  country_code        12 non-null     object
 12  description         1 non-null      object
 13  history             2 non-null      object
dtypes: int64(1), object(13)
memory usage: 18.9+ KB


## Limpieza de datos

Evidentemente, **una limpieza de los datos es necesaria**. Sólo las columnas `weight`, `height`, `id`, `name`, `life_span`, `reference_image_id`, e `image` están completas. Las demás poseen una cantidad variable de nulos, siendo peor en `description`, `history`, y `origin`. En particular, la necesaria columna `temperament` posee 4 nulos.

Desgraciadamente, al contener estas columnas problemáticas valores de tipo `object`, que son _no numéricos_, no hay forma sencilla de otorgarles valores de reemplazo que sean razonables. Por esto, con el fin de responder a las dos preguntas que guían este proyecto, intentaremos lo siguiente:

1. Crear un nuevo `DataFrame` que contenga sólo las columnas necesarias para responder a las preguntas de investigación.

2. Obviar los 4 registros cuyos valores en la columna `temperament` son nulos.

3. Eliminar posibles registros duplicados con `drop_duplicates()`.

In [7]:
# Paso 1
pesky_columns: list[str] = [
    'weight',
    'height',
    'bred_for',
    'breed_group',
    'origin',
    'reference_image_id',
    'image',
    'country_code',
    'description',
    'history'
]

clean_df: pd.DataFrame = pd.DataFrame(
    df.drop(pesky_columns, axis=1)
)
clean_df.head()

,id,name,life_span,temperament
0,1,Affenpinscher,10 - 12 years,"Stubborn, Curious, Playful, Adventurous, Activ..."
1,2,Afghan Hound,10 - 13 years,"Aloof, Clownish, Dignified, Independent, Happy"
2,3,African Hunting Dog,11 years,"Wild, Hardworking, Dutiful"
3,4,Airedale Terrier,10 - 13 years,"Outgoing, Friendly, Alert, Confident, Intellig..."
4,5,Akbash Dog,10 - 12 years,"Loyal, Independent, Intelligent, Brave"


In [8]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           172 non-null    int64 
 1   name         172 non-null    object
 2   life_span    172 non-null    object
 3   temperament  168 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.5+ KB


In [9]:
# Paso 2
clean_df = clean_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 0 to 171
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           168 non-null    int64 
 1   name         168 non-null    object
 2   life_span    168 non-null    object
 3   temperament  168 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


Posterior a esta sanitización urgente, debemos resolver el problema de tipado y contenido de la columna `life_span`, la cual es de tipo texto y parece mostrar la esperanza de vida mayormente en rangos, no en números exactos. Hay dos opciones:

1. Hemos de transformarla al tipo correcto (numérico) y transformar los rangos en promedios, para que sean posibles de computar en la generación de gráficos.

2. Generar una columna aparte _ad hoc_ con los promedios de esperanza de vida, conservando `life_span` como fuente de etiquetas para los gráficos.

In [10]:
clean_df['life_span'].value_counts()

life_span
10 - 12 years          28
12 - 15 years          28
12 - 14 years          26
10 - 13 years          12
10 - 14 years           7
8 - 10 years            6
12 - 16 years           5
15 years                5
7 - 10 years            3
11 - 13 years           3
13 - 15 years           3
6 - 8 years             2
12 years                2
10 years                2
10 - 11 years           2
14 - 15 years           2
13 - 16 years           2
14 - 16 years           2
12 – 14 years           2
10 - 15 years           2
13 – 15 years           2
12 - 13 years           2
9 – 14 years            2
15 - 18 years           1
12 - 18 years           1
10 - 18 years           1
12 - 16 Years years     1
14 - 18 years           1
13 - 17 years           1
15 - 20 years           1
8 - 12 years            1
12 – 15 years           1
10 – 16 years           1
10 – 15 years           1
9 - 11 years            1
11 - 15 years           1
11 – 14 years           1
8 - 15 years            1
8 

Tras pensarlo, optaré por la segunda opción, conservando la columna original `life_span`:

In [11]:
life_span_mean: pd.Series = (
    df['life_span'].str.replace('years', '').str.strip()
)
life_span_mean = life_span_mean.str.replace('Years', '').str.strip()
pre_sanitized_counts: pd.Series = life_span_mean.value_counts()  # important!
pre_sanitized_counts

life_span
10 - 12    29
12 - 15    28
12 - 14    27
10 - 13    12
10 - 14     7
12 - 16     6
8 - 10      6
15          5
7 - 10      3
11 - 13     3
13 - 15     3
10          2
14 - 15     2
12          2
6 - 8       2
9 – 14      2
10 - 11     2
13 - 16     2
14 - 16     2
12 – 14     2
10 - 15     2
13 – 15     2
12 - 13     2
12 – 15     2
15 - 18     1
12 - 18     1
10 - 18     1
18          1
14 - 18     1
13 - 17     1
15 - 20     1
8 - 12      1
10 – 16     1
10 – 15     1
9 - 11      1
11 - 15     1
11 – 14     1
8 - 15      1
8 – 15      1
11          1
13 – 14     1
Name: count, dtype: int64

Hay una mezcla de rangos y de números solitarios, por lo que no podremos pasarlos directamente a formato numérico. Hagamos una función para tratar los rangos y transformarlos a su promedio.

In [12]:
def process_life_span_data(value: str) -> float:
    signs: list[str] = ['-', '–']
    for sign in signs:
        if sign in value:
            low, high = map(float, value.split(sign))
            return (low + high) / 2
    else:
        return float(value)

In [13]:
life_span_mean = life_span_mean.apply(process_life_span_data)
sanitized_counts: pd.Series = life_span_mean.value_counts()
sanitized_counts

life_span
13.5    31
13.0    31
11.0    30
11.5    16
12.0    12
14.0    12
15.0     9
12.5     6
9.0      6
10.0     4
14.5     4
8.5      3
10.5     2
7.0      2
16.5     1
16.0     1
18.0     1
17.5     1
Name: count, dtype: int64

In [14]:
clean_df['life_span_mean'] = life_span_mean.astype('float64')
clean_df.head()

,id,name,life_span,temperament,life_span_mean
0,1,Affenpinscher,10 - 12 years,"Stubborn, Curious, Playful, Adventurous, Activ...",11.0
1,2,Afghan Hound,10 - 13 years,"Aloof, Clownish, Dignified, Independent, Happy",11.5
2,3,African Hunting Dog,11 years,"Wild, Hardworking, Dutiful",11.0
3,4,Airedale Terrier,10 - 13 years,"Outgoing, Friendly, Alert, Confident, Intellig...",11.5
4,5,Akbash Dog,10 - 12 years,"Loyal, Independent, Intelligent, Brave",11.0


In [15]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 0 to 171
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              168 non-null    int64  
 1   name            168 non-null    object 
 2   life_span       168 non-null    object 
 3   temperament     168 non-null    object 
 4   life_span_mean  168 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.9+ KB


In [16]:
clean_df.reset_index(drop=True, inplace=True)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              168 non-null    int64  
 1   name            168 non-null    object 
 2   life_span       168 non-null    object 
 3   temperament     168 non-null    object 
 4   life_span_mean  168 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.7+ KB


## 1. ¿Cuál es la esperanza de vida (en años) más frecuente entre todas las razas de perro?

En cierta forma, se ha logrado responder dicha pregunta durante el proceso de limpieza de datos, donde, a primera vista, la esperanza de vida más frecuente (según los datos categoriales de la columna `life_span`) pareciera estar en el rango de `10 - 12` años, con 29 casos.

In [17]:
pre_sanitized_distribution: go.Figure = go.Figure(go.Bar(
    x=pre_sanitized_counts.index,
    y=pre_sanitized_counts.values,
    marker=dict(
        color=pre_sanitized_counts.values,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Frecuencia')
    ),
    text=pre_sanitized_counts.values,
    textposition='outside',
    hovertemplate='<b>%{x} años:</b> %{y} casos <extra></extra>',
))

title: str = 'Distribución de la esperanza de vida*'
subtitle: str = '<br><sup>(datos ordenados desde el más frecuente)</sup>'

pre_sanitized_distribution.update_layout(
    title=(title + subtitle),
    xaxis_title='Esperanza de vida (años)',
    yaxis_title='Conteo de casos',
    xaxis = dict(
        tickmode='linear',
        tick0=0.5,
        dtick=0.75
    )
)

*Sin embargo, este gráfico es engañoso, y particularmente su dato más alto, ya que, por ejemplo, obvia las categorías de valor único que podrían estar dentro o fuera de ese rango.

Por ende, bajo un análisis más profundo, representado en la columna `life_span_mean` (donde los rangos han sido transformados a sus correspondientes medias aritméticas), se da un empate de frecuencias entre una esperanza de vida de `13.5` años y otra de `13` años, con 31 registros encontrados para cada caso.

In [28]:
sanitized_distribution: go.Figure = go.Figure(go.Bar(
    x=sanitized_counts.index,
    y=sanitized_counts.values,
    marker=dict(
        color=sanitized_counts.values,
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Frecuencia')
    ),
    text=sanitized_counts.values,
    textposition='outside',
    hovertemplate='<b>%{x} años:</b> %{y} casos <extra></extra>',
))

title: str = 'Distribución de la esperanza de vida'
subtitle: str = '<br><sup>(datos ordenados de menor a mayor)</sup>'

sanitized_distribution.update_layout(
    title=title + subtitle,
    xaxis_title='Esperanza de vida (años)',
    yaxis_title='Conteo de casos',
    xaxis=dict(
        tickmode='linear',
        dtick=0.5
    )
)

De esta manera, y simplificando aún más (promediando los valores del empate), se podría declarar que la esperanza de vida (en años) más frecuente entre todas las razas de perro, según los datos, se encuentra alrededor de los **13,25 años** (o lo que es lo mismo: 13 años y 3 meses).

## 2. ¿Cuál es el temperamento más frecuente entre todas las razas de perros?

Primero que todo, a primera vista, nuestro dataset `clean_df` nos indica que el temperamento de cada perro (en la columna `temperament`) no está compuesto por un solo valor descriptivo discreto, sino que generalmente por dos o más adjetivos.

In [19]:
grouped_temperaments: pd.Series = clean_df['temperament'].value_counts()
grouped_temperaments

temperament
Friendly, Alert, Reserved, Intelligent, Protective                                                                     2
Strong Willed, Stubborn, Friendly, Clownish, Affectionate, Loyal, Obedient, Intelligent, Courageous                    2
Sociable, Bubbly, Playful, Adaptable, Intelligent, Loving                                                              1
Playful, Docile, Fearless, Affectionate, Sweet-Tempered, Lively, Responsive, Easygoing, Gentle, Intelligent, Active    1
Energetic, Loyal, Intelligent, Trainable                                                                               1
                                                                                                                      ..
Affectionate, Sweet-Tempered, Lively, Alert, Playful, Happy                                                            1
Suspicious, Affectionate, Devoted, Reserved, Independent, Loving                                                       1
Friendly, Alert, Dig

Como se ve, al aplicar `value_counts()` a la columna `temperament`, obtenemos que los valores agrupados prácticamente no se repiten, salvo en dos oportunidades. Si se tomara como temperamento el conjunto de adjetivos, es claro que hay dos "temperamentos" más frecuentes:

- _Friendly, Alert, Reserved, Intelligent, Protective_

- _Strong Willed, Stubborn, Friendly, Clownish, Affectionate, Loyal, Obedient, Intelligent, Courageous_

Sin embargo, no creo que a esto se refiere la pregunta de este proyecto.

Observemos que hay adjetivos individuales como _"Loyal"_ o _"Independent"_ que se repiten continuamente entre las distintas agrupaciones de adjetivos por caso, y que sí tienen un valor estadístico significativo para nuestro estudio. Desde este punto, es claro que lo que se debe hacer es contar la ocurrencia de cada adjetivo en todos los casos (cuántas veces aparece _"Loyal"_, cuántas veces _"Independent"_, etc.). Por supuesto, hay muchas formas de resolver este problema. El desafío es encontrar la manera más eficiente de hacerlo.

In [20]:
adjectives: list[str] = []
for temperament in grouped_temperaments.index:
    adjectives.extend(temperament.split(', '))
adjectives = list(set(adjectives))
adjectives.sort()
adjectives

['Active',
 'Adaptable',
 'Adventurous',
 'Affectionate',
 'Aggressive',
 'Agile',
 'Alert',
 'Aloof',
 'Amiable',
 'Assertive',
 'Athletic',
 'Attentive',
 'Benevolent',
 'Boisterous',
 'Bold',
 'Bossy',
 'Brave',
 'Bright',
 'Bubbly',
 'Calm',
 'Cat-like',
 'Cautious',
 'Charming',
 'Cheerful',
 'Clever',
 'Clownish',
 'Companionable',
 'Composed',
 'Confident',
 'Cooperative',
 'Courageous',
 'Cunning',
 'Curious',
 'Determined',
 'Devoted',
 'Dignified',
 'Diligent',
 'Docile',
 'Dominant',
 'Dutiful',
 'Eager',
 'Easygoing',
 'Energetic',
 'Even Tempered',
 'Excitable',
 'Extroverted',
 'Faithful',
 'Familial',
 'Fast',
 'Fearless',
 'Feisty',
 'Fierce',
 'Friendly',
 'Fun-loving',
 'Gay',
 'Generous',
 'Gentle',
 'Good-natured',
 'Good-tempered',
 'Great-hearted',
 'Happy',
 'Hard-working',
 'Hardworking',
 'Hardy',
 'Independent',
 'Inquisitive',
 'Intelligent',
 'Joyful',
 'Keen',
 'Kind',
 'Lively',
 'Lovable',
 'Loving',
 'Loyal',
 'Merry',
 'Mischievous',
 'Obedient',
 'Opin

He aquí una lista de todos los adjetivos únicos. Aunque es verdad que muchos de ellos son sinónimos, los dejaremos como están para simplificar todo el proceso.

In [21]:
adjective_frequency: dict[str, int] = {}
for adjective in adjectives:
    for temperament in grouped_temperaments.index:
        if adjective in temperament:
            if adjective in adjective_frequency:
                adjective_frequency[adjective] += 1
            else:
                adjective_frequency[adjective] = 1

for adjective in adjective_frequency:
    print(f'{adjective}: {adjective_frequency[adjective]}')

Active: 25
Adaptable: 7
Adventurous: 1
Affectionate: 53
Aggressive: 1
Agile: 9
Alert: 51
Aloof: 8
Amiable: 2
Assertive: 4
Athletic: 4
Attentive: 6
Benevolent: 1
Boisterous: 1
Bold: 7
Bossy: 1
Brave: 3
Bright: 3
Bubbly: 1
Calm: 14
Cat-like: 1
Cautious: 2
Charming: 2
Cheerful: 6
Clever: 5
Clownish: 3
Companionable: 19
Composed: 1
Confident: 19
Cooperative: 1
Courageous: 15
Cunning: 1
Curious: 5
Determined: 1
Devoted: 11
Dignified: 8
Diligent: 1
Docile: 7
Dominant: 7
Dutiful: 2
Eager: 5
Easygoing: 3
Energetic: 29
Even Tempered: 11
Excitable: 1
Extroverted: 1
Faithful: 9
Familial: 5
Fast: 1
Fearless: 24
Feisty: 2
Fierce: 1
Friendly: 46
Fun-loving: 1
Gay: 3
Generous: 1
Gentle: 33
Good-natured: 4
Good-tempered: 1
Great-hearted: 1
Happy: 11
Hard-working: 2
Hardworking: 1
Hardy: 5
Independent: 21
Inquisitive: 4
Intelligent: 91
Joyful: 3
Keen: 8
Kind: 5
Lively: 20
Lovable: 2
Loving: 18
Loyal: 47
Merry: 1
Mischievous: 3
Obedient: 17
Opinionated: 1
Outgoing: 11
Patient: 7
People-Oriented: 1
Playf

El diccionario `adjective_frequency` nos indica la frecuencia encontrada para cada adjetivo. Los resultados están por orden alfabético, pero podemos reordenarlos por frecuencia de manera descendente. Posteriormente, procederemos a graficarlo:

In [25]:
adjective_frequency = dict(sorted(
        adjective_frequency.items(),
        key=lambda item: item[1],
        reverse=True
))
for adjective in adjective_frequency:
        print(f'{adjective}: {adjective_frequency[adjective]}')

Intelligent: 91
Affectionate: 53
Alert: 51
Loyal: 47
Friendly: 46
Playful: 37
Gentle: 33
Energetic: 29
Active: 25
Fearless: 24
Trainable: 23
Protective: 22
Independent: 21
Lively: 20
Companionable: 19
Confident: 19
Loving: 18
Obedient: 17
Courageous: 15
Calm: 14
Stubborn: 12
Devoted: 11
Even Tempered: 11
Happy: 11
Outgoing: 11
Reserved: 11
Strong: 10
Agile: 9
Faithful: 9
Quiet: 9
Sociable: 9
Spirited: 9
Sweet-Tempered: 9
Aloof: 8
Dignified: 8
Keen: 8
Adaptable: 7
Bold: 7
Docile: 7
Dominant: 7
Patient: 7
Responsive: 7
Sensitive: 7
Strong Willed: 7
Attentive: 6
Cheerful: 6
Quick: 6
Watchful: 6
Clever: 5
Curious: 5
Eager: 5
Familial: 5
Hardy: 5
Kind: 5
Reliable: 5
Steady: 5
Tenacious: 5
Assertive: 4
Athletic: 4
Good-natured: 4
Inquisitive: 4
Powerful: 4
Brave: 3
Bright: 3
Clownish: 3
Easygoing: 3
Gay: 3
Joyful: 3
Mischievous: 3
Proud: 3
Self-assured: 3
Self-confidence: 3
Territorial: 3
Trusting: 3
Vocal: 3
Amiable: 2
Cautious: 2
Charming: 2
Dutiful: 2
Feisty: 2
Hard-working: 2
Lovable: 2


In [26]:
temperament_distribution: go.Figure = go.Figure(go.Bar(
    orientation='h',
    x=list(adjective_frequency.values()),
    y=list(adjective_frequency.keys()),
    marker=dict(
        color=list(adjective_frequency.values()),
        colorscale='Viridis',
        showscale=False,
        colorbar=None
    ),
    text=list(adjective_frequency.values()),
    textposition='outside',
    hovertemplate='<b>%{y}:</b> %{x} casos <extra></extra>'
))

temperament_distribution.update_layout(
    autosize=False,
    height=4096,
    title='Distribución de los temperamentos',
    xaxis_title='Conteo de casos',
    yaxis_title='Temperamento',
    yaxis=dict(
        autorange='reversed',
        tickmode='linear',
        dtick=0.5
    )
)

Como se ve, el adjetivo **"Intelligent"** (inteligente) es, por lejos, el más frecuente para describir el temperamento de un perro, al estar presente en 91 casos. Le siguen adjetivos como "Affectionate" (afectuoso) y "Alert" (en alerta), con 53 y 51 ocurrencias, respectivamente.